## Web of Science API
This code is designed to interact with the WOS API using "premium" or "lite" access protocols.

-    `wos.py` contains code for interacting with the API directly.
-    `woscalls.py` includes calls made to the `WOS` class
-    `metawos.py` can be used to extract metadata from search results.
-    `buildsearch.py` builds search strings from data supplied in a certain style of tsv file.

### Dependencies

-    [lxml](http://lxml.de/) for xml parsing
-    [suds](https://fedorahosted.org/suds/wiki/Documentation) for SOAP API interaction.

### The API Class

It's possible to work with the API class, `WOS`, directly using syntax like the following:

In [13]:
from wos import Wos
wos = Wos(client="Lite")

This will initiate the search client but not yet run any API calls. There are two options for the "client" keyword argument: "Search" and "Lite". The Lite API should be available to all institutions that subscribe to at least the Web of Knowledge Core Collection. Contact your representative to have access opened to your IP address, if it isn't already. [More details here](http://wokinfo.com/products_tools/products/related/webservices/). The Lite API provides basic search and basic metadata retrieval functionality, but _not_ citation retrieval.

The "Search" client is an available at an additional cost, and does provide access to both forward and backward citation data. Access is available on a project basis, or as a yearly subscription. I would recommend first requesting a trial period to work with the API.

In [14]:
wos.authorize()

Search client authorized.


The `authorize` function attempts to authenticate based on IP address. If successful, an authorization token will be attached to all future requests in the session. 

In [15]:
wos.query_parameters('AU=(Peiretti AND Palmegiano) AND SO=(Animal Feed Science and Technology) AND PY=2004', database_id="WOK")

(queryParameters){
   databaseId = "WOK"
   userQuery = "AU=(Peiretti AND Palmegiano) AND SO=(Animal Feed Science and Technology) AND PY=2004"
   editions[] = <empty>
   symbolicTimeSpan = None
   timeSpan = 
      (timeSpan){
         begin = "1900-01-01"
         end = "2015-10-01"
      }
   queryLanguage = "en"
 }

First establish the `query_parameters` object, which should include a query string along with, optionally, a set of parameters. Details on the structure of the query can be found in the API documentation (which can be requested from ThomsonReutuers, but which I'm also making available [here](https://www.msu.edu/~higgi135/WebServicesLiteguide.pdf) (in possibly an outdated version).  

Parameters can be provided as the following keyword arguments:

- **time_begin (str)** -- date in YYYY-MM-DD format.
- **time_end (str)** -- date in YYYY-MM-DD format.
- **database_id (str)** -- from the WOS set of database abbreviations. "WOS" correpsonds to the WOS core collection.
- **query_language (str)** -- "en" the only currently allowed value.
- **symbolic_timespan (str)** -- a human-readable timespan, e.g. "4weeks", must be null if time_begin and time_end used.
- **editions (list)** -- TODO list of sub-components of the selected database to use.

In [16]:
wos.retrieve_parameters()

(retrieveParameters){
   firstRecord = 1
   count = 100
   sortField[] = <empty>
 }

The `retrieve_parameters` allow for some control of the data that is returned.  

- **first_record (int)** -- The number of the first record to return in the search.
- **count (int)** -- Number of records to return (maximum 100).
- **sort_field (list)** -- TODO Field to sort by (should be WOS field abbreviation).

In [17]:
wos.search(wos.qp, wos.rp)

Found 1 Results for AU=(Peiretti AND Palmegiano) AND SO=(Animal Feed Science and Technology) AND PY=2004


(searchResults){
   queryId = "1"
   recordsFound = 1
   recordsSearched = 198371943
   records[] = 
      (liteRecord){
         uid = "WOS:000224567700011"
         title[] = 
            (labelValuesPair){
               label = "Title"
               value[] = 
                  "Chemical composition, organic matter digestibility and fatty acid content of evening primrose (Oenothera paradoxa) during its growth cycle",
            },
         source[] = 
            (labelValuesPair){
               label = "Issue"
               value[] = 
                  "3-4",
            },
            (labelValuesPair){
               label = "Pages"
               value[] = 
                  "293-299",
            },
            (labelValuesPair){
               label = "Published.BiblioDate"
               value[] = 
                  "OCT 15",
            },
            (labelValuesPair){
               label = "Published.BiblioYear"
               value[] = 
                  "2004",
   

Run the search by calling the `search` function with the query parameters and retrieve parameters objects as arguments (`wos.qp` and `wos.rp` respectively). 

Results can be found in the `wos.search_results.records` object, if any results were returned. More generally, `wos.search_results` can be used to find info about the response, including number of results.  

Additional methods can be used to get cited references as well as citing articles if the "Search" client is used.

In [2]:
from wos import Wos
wos = Wos(client="Lite")
wos.authorize()
wos.query_parameters('AU=(Peiretti AND Palmegiano) AND SO=(Animal Feed Science and Technology) AND PY=2004', database_id="WOK")
wos.retrieve_parameters(view_field=["title", "name"])
wos.search(wos.qp, wos.rp)

Search client authorized.
Found 1 Results for AU=(Peiretti AND Palmegiano) AND SO=(Animal Feed Science and Technology) AND PY=2004


(searchResults){
   queryId = "1"
   recordsFound = 1
   recordsSearched = 198821686
   records[] = 
      (liteRecord){
         uid = "WOS:000224567700011"
         title[] = 
            (labelValuesPair){
               label = "Title"
               value[] = 
                  "Chemical composition, organic matter digestibility and fatty acid content of evening primrose (Oenothera paradoxa) during its growth cycle",
            },
         source[] = 
            (labelValuesPair){
               label = "Issue"
               value[] = 
                  "3-4",
            },
            (labelValuesPair){
               label = "Pages"
               value[] = 
                  "293-299",
            },
            (labelValuesPair){
               label = "Published.BiblioDate"
               value[] = 
                  "OCT 15",
            },
            (labelValuesPair){
               label = "Published.BiblioYear"
               value[] = 
                  "2004",
   

The above code should return 1 result and can be used as a test to ensure code is working properly.

The below search operation run in order to retrieve results for a complex search.

In [ ]:
from wos import Wos
wos = Wos(client="Lite")
wos.authorize()
wos.query_parameters('TS=(("Health Behavio$r" OR "Health Communication" OR "Environmental Health" OR "Environmental Conservation" OR "Animal health" OR "Animal Well Being") AND Intervention AND ("Mobile Technologies" OR "SMS" OR "Text messaging" OR "Texting" OR "Cellphones" OR "Smartphones" OR "Smartphone app*" OR "mobile devices" OR "wireless devices" OR "Smart devices" OR "Connected devices" OR "Tracking" OR Monitoring OR Surveillance OR Tablets))', database_id="WOK")
wos.retrieve_parameters()
wos.search(wos.qp, wos.rp)

In [6]:
wos._get_metadata("References", "search_results")

Somewhat messy code to transform WOS results into RIS format for upload to Covidence.

In [ ]:
import codecs
with codecs.open("intervention.ris", "w", "utf-8") as output:
    for result in wosc.wos.metadata_collection["search_results"]:
        output.write(u"TY  - JOUR\n")
        if "authors" in result:
            for index, author in enumerate(result["authors"][0]["value"]):
                output.write(u"A{0}  - {1}\n".format(index + 1, unicode(author)))
        if "title" in result:
            for index, title in enumerate(result["title"][0]["value"]):
                output.write(u"T{0}  - {1}\n".format(index + 1, unicode(title)))           

        if "keywords" in result:
            for index, keyword in enumerate(result["keywords"][0]["value"]):
                output.write(u"KW  - {0}\n".format(unicode(keyword)))    
        if "other" in result:
            for i in result["other"]:
                if "Doi" in i["label"]:
                    doi = i["value"][0]
                    output.write(u"DO  - {0}\n".format(doi))
                if "Issn" in i["label"]:
                    issn = i["value"][0]
                    output.write(u"SN  - {0}\n".format(issn))
        if "source" in result:
            for i in result["source"]:
                if "Pages" in i["label"]:
                    try:
                        startpage = i["value"][0].split("-")[0]
                        endpage = i["value"][0].split("-")[1]
                        output.write(u"SP  - {0}\n".format(startpage))
                        output.write(u"EP  - {0}\n".format(endpage))
                    except:
                        # print "Falied on page numbers for {0}".format(i["value"])
                if "SourceTitle" in i["label"]:
                    sourcetitle = unicode(i["value"][0])
                    output.write(u"JF  - {0}\n".format(sourcetitle))
                if "BiblioYear" in i["label"]:
                    output.write(u"PY  - {0}\n".format(i["value"][0]))
                if "Volume" in i["label"]:
                    output.write(u"VL  - {0}\n".format(i["value"][0]))
                if "Issue" in i["label"]:
                    output.write(u"IS  - {0}\n".format(i["value"][0]))
        output.write(u"ER  - \n")

### Automating Searches

The `WosCalls` class provides a means of interacting with the API 1 level up. That is, lists of search strings or sets of search parameters can be provided to run in batch. *This functionality is still work in progress*.

In [19]:
from woscalls import WosCalls
wosc = WosCalls(search_queries=bs.searches, database_id="WOK")
wosc.get_all_search_results()

Search client authorized.
Found 1 Results for AU=(Lambertsen) AND PY=1966 AND SO=(Acta Agric* Scand*)
Found 1 Results for AU=(Bentes) AND PY=1986 AND SO=(Acta Amazonica)
Found 1 Results for AU=(Maia) AND PY=1978 AND SO=(Acta Amazonica)
Found 0 Results for AU=(Loth) AND PY=1991 AND SO=(Agrochimica)
Found 1 Results for AU=(Jellum AND Powell) AND PY=1971 AND SO=(Agron* J*)
Found 5 Results for AU=(Bertoni) AND PY=1994 AND SO=(An* Asoc* Quim* Argent*)
Found 0 Results for AU=(Balnchini) AND PY=1981 AND SO=(Anal* Chem*)
Found 1 Results for AU=(Peiretti AND Palmegiano AND Masoero) AND PY=2004 AND SO=(Animal Feed Science and Technology)
Found 0 Results for AU=(Adhikari) AND PY=1991 AND SO=(Bangladesh J* Sci* Ind* Res*)
Found 0 Results for AU=(Serrano AND Guzm?n) AND PY=1994 AND SO=(Biochem* Systemat* Ecology)
Process complete.
Returned 10 records


In [ ]:
from woscalls import WosCalls
wosc = WosCalls(search_term_sets=bs.search_terms, database_id="WOK")
wosc.find_exact_match()

The `WosCalls` class is additionally a place to house content-specific methods built on `Wos`. See `run_phylo_process` method as it develops.

### Additional Information

The `BuildSearch` class is currently quite content specific but could in principle be broadened to allow for automatically generating searches from data in other formats, such as CSV or JSON. Currently the algorithm below assumes a very specific data structure to work.

In [ ]:
from buildsearch import OhlroggeSearch
bs = OhlroggeSearch("data/ohlrogge/ohlrogge_test_10.txt")
bs.make_search_list() # from here the object bs.searches can be used in WosCalls()

In [1]:
from buildsearch import OhlroggeSearch
bs = OhlroggeSearch("ohlrogge-data.tsv")
bs.make_search_dict() # from here the object bs.search_terms can be used in WosCalls()

File loaded.
Extracted 2896 lines


In [2]:
bs.search_terms

[{'author': '(Lambertsen)',
  'id': u'11702',
  'page': u'213',
  'plant_count': 8,
  'query': u'AU=(Lambertsen) AND PY=1966 AND SO=(Acta Agricultarae Scandinavica)',
  'result_count': 7,
  'source': '(Acta Agricultarae Scandinavica)',
  'volume': u'16',
  'year': u'1966'},
 {'author': '(Lambertsen)',
  'id': u'11703',
  'page': u'213',
  'plant_count': 8,
  'query': u'AU=(Lambertsen) AND PY=1966 AND SO=(Acta Agricultarae Scandinavica)',
  'result_count': 7,
  'source': '(Acta Agricultarae Scandinavica)',
  'volume': u'16',
  'year': u'1966'},
 {'author': '(Lambertsen)',
  'id': u'11704',
  'page': u'213',
  'plant_count': 8,
  'query': u'AU=(Lambertsen) AND PY=1966 AND SO=(Acta Agricultarae Scandinavica)',
  'result_count': 7,
  'source': '(Acta Agricultarae Scandinavica)',
  'volume': u'16',
  'year': u'1966'},
 {'author': '(Lambertsen)',
  'id': u'11705',
  'page': u'213',
  'plant_count': 8,
  'query': u'AU=(Lambertsen) AND PY=1966 AND SO=(Acta Agriculturae Scandinavica)',
  'resul

The `bs.searches` object contains a list of searches, suitable to pass as an argument in the `WosCalls` class.

In [ ]:
import codecs

with codecs.open("data/updated_ohlrogge_data_20151017.tsv", "w", "utf-8") as f:
    headings = wosc.article_data["11542"][0].keys()
    f.write("\t".join(headings)+"\n")
    for key in wosc.article_data:
        if isinstance(wosc.article_data[key], dict):
            record = wosc.article_data[key]
            record_data = []
            for h in headings:
                data_point = record.get(h, "None")
                if isinstance(data_point, list):
                    record_data.append(data_point[0] if len(data_point)>0 else"None")
                else:
                    record_data.append(data_point)
            f.write("\t".join([unicode(i) for i in record_data]) + "\n")
        else:
            for record in wosc.article_data[key]:
                record_data = []
                for h in headings:
                    data_point = record.get(h, "None")
                    if isinstance(data_point, list):
                        record_data.append(data_point[0] if len(data_point)>0 else"None")
                    else:
                        record_data.append(data_point)
                f.write("\t".join([unicode(i) for i in record_data]) + "\n")

#### Get in touch!

If I can be of help in using this code, or if you have suggestions for improvement, please do contact me.